# Building ANCOR Dataset

In [ ]:
from french_crs.ancor2dataset import dataset_builder


for method in ["balanced", "representative", "window"]:
    for subcorpus in [["INDIRECTE"],["DIRECTE"],["ANAPHORE"]]:
        dataset_params={
            # There are other three possible strategies: (1)"balanced", (2)"representative" and (3)"window"
            "strategy" : method,
            "ancor_corpus_path" : "../DISTRIB_ANCOR/",
            # All possibilities are: ["corpus_OTG","corpus_UBS","corpus_ESLO" ,"corpus_ESLO_CO2"]
            "sub_corpus_filter" : ["corpus_OTG","corpus_UBS","corpus_ESLO" ,"corpus_ESLO_CO2"], 
            # All possibilities are: ["DIRECTE", "INDIRECTE", "ANAPHORE"]
            "coreference_type_filter" : subcorpus, 
            "dataset_output_folder" : "../datasets/",
            # This will be used only in window strategy
            "window_size" : 5 
        }

        dataset = dataset_builder(**dataset_params)
        dataset.build_dataset(file_analysis_alert=True)
        dataset.merge_dataset(delete_original_after_merge=False)

# Dataset Splitting

In [ ]:
# Two ways of Training: (1) commented codes down with parameter.
#                       (2) uncommented with mentioned explicity the config file.

from french_crs.model_training import dataset_splitter

# ds1=dataset_splitter("../datasets/balanced_Full/balanced_Full_Merged.xlsx",
#                     "../datasets/balanced_Full/balanced_Full_Train.xlsx",
#                     "../datasets/balanced_Full/balanced_Full__Test.xlsx",
#                       split_config_json="../datasets/split_config.json")

# ds1.dataset_splitter_by_file(lower_rate=0.20,upper_rate=0.21, files_num=30)


ds2=dataset_splitter("../datasets/window_INDIRECTE/window_INDIRECTE_Merged.xlsx",
                    "../datasets/window_INDIRECTE/window_INDIRECTE_Train.xlsx",
                    "../datasets/window_INDIRECTE/window_INDIRECTE_Test.xlsx",
                      split_config_json="../datasets/split_config.json")

dict_files=ds2.dataset_splitter_by_json_config()


# Model Training

In [ ]:
from model_training import model_trainer

model=model_trainer( "./ds_window_all/corpus_ANCOR_Window_ANAPHORE_Train.xlsx",
                     "./ds_window_5/corpus_ANCOR_Window_5_Test.xlsx",
                     "./ds_window_all/corpus_ANCOR_Window_Full_ANAPHORE_over_Test_Pred.xlsx",
                     "IS_CO_REF",
                     "IS_CO_REF"
                   )

model.columns_drop_list = ['m1_DEF', 'm2_DEF', 'ID_DEF',
                           'DISTANCE_MENTION','DISTANCE_WORD',
                           'DISTANCE_CHAR']

model.convert_columns_to_numeric()
model.train_model_random_forest(model_name="./Optimized_Models/Random_Forest_ANCOR_Window_ANAPHORE.model",max_depth=10, random_state=0,n_estimators=250)

# Model Testing

In [ ]:
from model_training import model_tester

model_parameter={
                "model_name" : "./Models/Random_Forest_(Normal)_OTG_Neg_90_Pos_10.model",
                "input_file" : "./Datasets/corpus_ALL_Window_30_Test.xlsx",
                "output_file" : "./Datasets/corpus_ALL_Window_30_Test_Called_Seperately.xlsx",
                "column_gold" : "IS_CO_REF",
                "column_outcome" : "Prediction",
                "threshold" : 0.5
                }

model=model_tester(**model_parameter)
model.apply_model_to_dataset()